In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import gzip
import json
import sqlalchemy
import datetime
import math

In [2]:
rec_data = [json.loads(line) for line in gzip.open("receipts.json.gz", "r")]
rec_df = pd.DataFrame(rec_data)
rec_df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


#### Some of the receipts have no items on them, as shown below. We have to drop the empty receipts.

In [3]:
rec_df.iloc[71]

_id                        {'$oid': '5ff475820a7214ada10005cf'}
bonusPointsEarned                                           NaN
bonusPointsEarnedReason                                     NaN
createDate                             {'$date': 1609856386000}
dateScanned                            {'$date': 1609856386000}
finishedDate                                                NaN
modifyDate                             {'$date': 1609856386000}
pointsAwardedDate                                           NaN
pointsEarned                                                NaN
purchaseDate                                                NaN
purchasedItemCount                                          NaN
rewardsReceiptItemList                                      NaN
rewardsReceiptStatus                                  SUBMITTED
totalSpent                                                  NaN
userId                                 5a43c08fe4b014fd6b6a0612
Name: 71, dtype: object

#### We are dropping empty receipts and then reformating IDs

In [4]:
rec_clean_df = rec_df.dropna(subset=['rewardsReceiptItemList']).copy()
order_ids = [i_d['$oid'] for i_d in rec_clean_df._id]
rec_clean_df['receipt_id'] = order_ids
rec_cl_df = rec_clean_df.drop(columns=['rewardsReceiptItemList','_id'])
rec_cl_df.head()

,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,receipt_id
0,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575
1,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b
2,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5,{'$date': 1609632000000},1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a
3,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f
4,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561


#### Compare size of receipts dataframes without empty itemsand and with empty items

In [5]:
print(f'This is DF with all receipts: {rec_df.shape}, \n\
This is DF without empty receipts: {rec_clean_df.shape}')

This is DF with all receipts: (1119, 15), 
This is DF without empty receipts: (679, 16)


#### Now, lets reformat the date fields

In [6]:
def extract_dates(df,column):
    new_column = []
    for dd in df[column]:
        if type(dd) == dict:
            new_dd = math.trunc(dd['$date']/1000)
            new_dd = pd.to_datetime(new_dd, unit='s')
            new_column.append(new_dd)
        else:
            new_column.append(dd)
    df[column] = new_column
    return df

In [7]:
rec_formatted_df = extract_dates(rec_cl_df, 'createDate')
rec_formatted_df = extract_dates(rec_formatted_df, 'dateScanned')
rec_formatted_df = extract_dates(rec_formatted_df, 'finishedDate')
rec_formatted_df = extract_dates(rec_formatted_df, 'modifyDate')
rec_formatted_df = extract_dates(rec_formatted_df, 'pointsAwardedDate')
rec_formatted_df = extract_dates(rec_formatted_df, 'purchaseDate')

In [8]:
rec_formatted_df.head()

,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,receipt_id
0,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575
1,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b
2,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a
3,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f
4,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561


#### Checking for datatypes and converting if needed

In [9]:
rec_formatted_df.dtypes

bonusPointsEarned                 float64
bonusPointsEarnedReason            object
createDate                 datetime64[ns]
dateScanned                datetime64[ns]
finishedDate               datetime64[ns]
modifyDate                 datetime64[ns]
pointsAwardedDate          datetime64[ns]
pointsEarned                       object
purchaseDate               datetime64[ns]
purchasedItemCount                float64
rewardsReceiptStatus               object
totalSpent                         object
userId                             object
receipt_id                         object
dtype: object

In [10]:
rec_formatted_df = rec_formatted_df.astype({'pointsEarned': 'float64',
                                            'purchasedItemCount': 'Int64',
                                            'totalSpent': 'float64'})

#### Now let's see our data types :

In [11]:
rec_formatted_df.dtypes

bonusPointsEarned                 float64
bonusPointsEarnedReason            object
createDate                 datetime64[ns]
dateScanned                datetime64[ns]
finishedDate               datetime64[ns]
modifyDate                 datetime64[ns]
pointsAwardedDate          datetime64[ns]
pointsEarned                      float64
purchaseDate               datetime64[ns]
purchasedItemCount                  Int64
rewardsReceiptStatus               object
totalSpent                        float64
userId                             object
receipt_id                         object
dtype: object

#### Creating dataframe with items and corresponding receipt_id for our OrderItemList SQL table

In [12]:
order_item_list_data = [x for x in rec_clean_df.rewardsReceiptItemList]
order_ids = [i_d['$oid'] for i_d in rec_clean_df._id]

In [13]:
for index, order in enumerate(order_item_list_data):
    try:
        for item in order:
            item['receipt_id'] = order_ids[index]
    except TypeError:
        print(f'We have cougth missing value at position {index}')

#### Now , Let's chech if all items have receipt_id on them:

In [14]:
order_item_list_data

[[{'barcode': '4011',
   'description': 'ITEM NOT FOUND',
   'finalPrice': '26.00',
   'itemPrice': '26.00',
   'needsFetchReview': False,
   'partnerItemId': '1',
   'preventTargetGapPoints': True,
   'quantityPurchased': 5,
   'userFlaggedBarcode': '4011',
   'userFlaggedNewItem': True,
   'userFlaggedPrice': '26.00',
   'userFlaggedQuantity': 5,
   'receipt_id': '5ff1e1eb0a720f0523000575'}],
 [{'barcode': '4011',
   'description': 'ITEM NOT FOUND',
   'finalPrice': '1',
   'itemPrice': '1',
   'partnerItemId': '1',
   'quantityPurchased': 1,
   'receipt_id': '5ff1e1bb0a720f052300056b'},
  {'barcode': '028400642255',
   'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
   'finalPrice': '10.00',
   'itemPrice': '10.00',
   'needsFetchReview': True,
   'needsFetchReviewReason': 'USER_FLAGGED',
   'partnerItemId': '2',
   'pointsNotAwardedReason': 'Action not allowed for user and CPG',
   'pointsPayerId': '5332f5fbe4b03c9a25efd0ba',
   'preventTargetGapPoint

#### Now, when every item in a list have ordet_id we can split them and create order_item_list_df

In [15]:
order_items_list = []
for order in order_item_list_data:
    for item in order:
        order_items_list.append(item)
        
order_item_list_df = pd.DataFrame(order_items_list)
items_formatted_df = order_item_list_df
items_formatted_df

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,receipt_id,needsFetchReviewReason,pointsNotAwardedReason,pointsPayerId,rewardsGroup,rewardsProductPartnerId,userFlaggedDescription,originalMetaBriteBarcode,originalMetaBriteDescription,brandCode,competitorRewardsGroup,discountedItemPrice,originalReceiptItemText,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,5ff1e1eb0a720f0523000575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,10.00,1.0,5ff1e1bb0a720f052300056b,USER_FLAGGED,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,5ff1e1ee0a7214ada100056f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,603cc2bc0a720fde100003e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,NaN,NaN,603cc0630a720fde100003e6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,mueller austria hypergrind precision electric ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,603cc0630a720fde100003e6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,NaN,NaN,603ce7100a7217c72c000405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,mueller austria hypergrind precision electric ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


#### Checking for data Types and converting is needed

In [16]:
items_formatted_df.dtypes

barcode                                object
description                            object
finalPrice                             object
itemPrice                              object
needsFetchReview                       object
partnerItemId                          object
preventTargetGapPoints                 object
quantityPurchased                     float64
userFlaggedBarcode                     object
userFlaggedNewItem                     object
userFlaggedPrice                       object
userFlaggedQuantity                   float64
receipt_id                             object
needsFetchReviewReason                 object
pointsNotAwardedReason                 object
pointsPayerId                          object
rewardsGroup                           object
rewardsProductPartnerId                object
userFlaggedDescription                 object
originalMetaBriteBarcode               object
originalMetaBriteDescription           object
brandCode                         

In [17]:
items_formatted_df = items_formatted_df.astype({'barcode': 'object',
                                                 'finalPrice': 'float64',
                                                 'itemPrice': 'float64',
                                                 'needsFetchReview': 'bool',
                                                 'partnerItemId': 'object',
                                                 'preventTargetGapPoints': 'bool',
                                                 'quantityPurchased': 'Int64',
                                                 'userFlaggedBarcode': 'object',
                                                 'userFlaggedNewItem': 'bool',
                                                 'userFlaggedPrice': 'float64',
                                                 'userFlaggedQuantity' :'Int64',
                                                 'receipt_id': 'object',
                                                 'needsFetchReviewReason': 'object',
                                                 'pointsNotAwardedReason': 'object',
                                                 'pointsPayerId': 'object',
                                                 'discountedItemPrice': 'float64',
                                                 'originalMetaBriteQuantityPurchased': 'Int64',
                                                 'pointsEarned': 'float64',
                                                 'targetPrice': 'float64',
                                                 'originalFinalPrice': 'float64'
                                                 })

In [18]:
items_formatted_df.dtypes

barcode                                object
description                            object
finalPrice                            float64
itemPrice                             float64
needsFetchReview                         bool
partnerItemId                          object
preventTargetGapPoints                   bool
quantityPurchased                       Int64
userFlaggedBarcode                     object
userFlaggedNewItem                       bool
userFlaggedPrice                      float64
userFlaggedQuantity                     Int64
receipt_id                             object
needsFetchReviewReason                 object
pointsNotAwardedReason                 object
pointsPayerId                          object
rewardsGroup                           object
rewardsProductPartnerId                object
userFlaggedDescription                 object
originalMetaBriteBarcode               object
originalMetaBriteDescription           object
brandCode                         

#### Check if we have same number of receipts here as in receipts DF

In [19]:
items_formatted_df.groupby(by='receipt_id').count().shape[0]

679

#### Now let's format users dataframe

In [20]:
user_data = [json.loads(line) for line in gzip.open("users.json.gz", "r")]
user_df = pd.DataFrame(user_data)
user_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [21]:
users_id_list = [i['$oid'] for i in user_df._id]

In [22]:
user_df._id = users_id_list
user_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [23]:
user_formatted_df = extract_dates(user_df,'createdDate')
user_formatted_df = extract_dates(user_formatted_df,'lastLogin')
user_formatted_df = user_formatted_df.rename(columns={'_id':'user_id'})
print (f'Checking for unique user ids for our DB later on: \n \
Number of rows: {user_formatted_df.shape[0]}\n \
Number of unique ids: {user_formatted_df.nunique()[0]}')

Checking for unique user ids for our DB later on: 
 Number of rows: 495
 Number of unique ids: 212


#### Let's get rid of duplicates

In [24]:
user_formatted_df = user_formatted_df.drop_duplicates(subset='user_id')

#### Formating data types:

In [25]:
user_formatted_df.dtypes

user_id                 object
active                    bool
createdDate     datetime64[ns]
lastLogin       datetime64[ns]
role                    object
signUpSource            object
state                   object
dtype: object

#### Now, let's format brands DF

In [26]:
br_data = [json.loads(line) for line in gzip.open("brands.json.gz", "r")]
br_df = pd.json_normalize(br_data)
br_formatted_df = br_df.rename(columns={'_id.$oid': 'brand_id', 'cpg.$id.$oid': 'cpg_id', 'cpg.$ref': 'cpg_reference'})
br_formatted_df.head()

,barcode,category,categoryCode,name,topBrand,brand_id,cpg_id,cpg_reference,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


#### Formating data types:

In [27]:
br_formatted_df.dtypes

barcode          object
category         object
categoryCode     object
name             object
topBrand         object
brand_id         object
cpg_id           object
cpg_reference    object
brandCode        object
dtype: object

In [28]:
br_formatted_df = br_formatted_df.astype({'topBrand': 'bool'})

In [29]:
br_formatted_df.dtypes

barcode          object
category         object
categoryCode     object
name             object
topBrand           bool
brand_id         object
cpg_id           object
cpg_reference    object
brandCode        object
dtype: object

#### These are our 4 Data Frames for SQL tables
    rec_formatted_df
    items_formatted_df
    user_formatted_df
    br_formatted_df

In [30]:
engine = sqlalchemy.create_engine('sqlite:///fetch_rewards.db', echo = True)

In [31]:
rec_formatted_df.to_sql('temp_receipts', con=engine, if_exists='replace')
items_formatted_df.to_sql('temp_items', con=engine, if_exists='replace')
user_formatted_df.to_sql('temp_users',con=engine, if_exists='replace')
br_formatted_df.to_sql('temp_brands',con=engine, if_exists='replace')

2021-08-15 23:10:26,715 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("temp_receipts")
2021-08-15 23:10:26,716 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-15 23:10:26,717 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("temp_receipts")
2021-08-15 23:10:26,717 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-15 23:10:26,719 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-15 23:10:26,720 INFO sqlalchemy.engine.Engine 
CREATE TABLE temp_receipts (
	"index" BIGINT, 
	"bonusPointsEarned" FLOAT, 
	"bonusPointsEarnedReason" TEXT, 
	"createDate" DATETIME, 
	"dateScanned" DATETIME, 
	"finishedDate" DATETIME, 
	"modifyDate" DATETIME, 
	"pointsAwardedDate" DATETIME, 
	"pointsEarned" FLOAT, 
	"purchaseDate" DATETIME, 
	"purchasedItemCount" BIGINT, 
	"rewardsReceiptStatus" TEXT, 
	"totalSpent" FLOAT, 
	"userId" TEXT, 
	receipt_id TEXT
)


2021-08-15 23:10:26,721 INFO sqlalchemy.engine.Engine [no key 0.00091s] ()
2021-08-15 23:10:26,725 INFO sqlalchemy.engine.Engine CREAT

2021-08-15 23:10:26,981 INFO sqlalchemy.engine.Engine COMMIT
2021-08-15 23:10:26,992 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("temp_users")
2021-08-15 23:10:26,993 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-15 23:10:26,993 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("temp_users")
2021-08-15 23:10:26,994 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-15 23:10:26,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-15 23:10:26,997 INFO sqlalchemy.engine.Engine 
CREATE TABLE temp_users (
	"index" BIGINT, 
	user_id TEXT, 
	active BOOLEAN, 
	"createdDate" DATETIME, 
	"lastLogin" DATETIME, 
	role TEXT, 
	"signUpSource" TEXT, 
	state TEXT
)


2021-08-15 23:10:26,997 INFO sqlalchemy.engine.Engine [no key 0.00060s] ()
2021-08-15 23:10:27,001 INFO sqlalchemy.engine.Engine CREATE INDEX ix_temp_users_index ON temp_users ("index")
2021-08-15 23:10:27,001 INFO sqlalchemy.engine.Engine [no key 0.00047s] ()
2021-08-15 23:10:27,005 INFO sqlalchemy.engine.Engine CO

#### Let's make runnig sql queries easy from Jupyter Notebook inline. We'll use "ipython-sql"

In [32]:
%load_ext sql
%sql sqlite:///fetch_rewards.db

#### Turning ON foreign keys in SQLite.

In [33]:
%%sql
PRAGMA foreign_keys = ON;

 * sqlite:///fetch_rewards.db
Done.


[]

In [34]:
%%sql
PRAGMA foreign_keys;

 * sqlite:///fetch_rewards.db
Done.


foreign_keys
1


#### Now create new tables from the sql statements based on our temporay tables. But here we define Prmary keys and Foreign Keys. We have to do this workaround , baceuse SQLite doesn't support ALTER constraints on existing tables.

In [35]:
%%sql
CREATE TABLE IF NOT EXISTS receipts (
        "index" BIGINT,
        "bonusPointsEarned" FLOAT,
        "bonusPointsEarnedReason" TEXT,
        "createDate" DATETIME,
        "dateScanned" DATETIME,
        "finishedDate" DATETIME,
        "modifyDate" DATETIME,
        "pointsAwardedDate" DATETIME,
        "pointsEarned" FLOAT,
        "purchaseDate" DATETIME,
        "purchasedItemCount" BIGINT,
        "rewardsReceiptStatus" TEXT,
        "totalSpent" FLOAT,
        "userId" TEXT,
        receipt_id TEXT NOT NULL PRIMARY KEY,
        FOREIGN KEY (userId) REFERENCES users (user_id) 
);
CREATE INDEX IF NOT EXISTS ix_receipts_index ON receipts ("index");


CREATE TABLE IF NOT EXISTS users (
        "index" BIGINT,
        user_id TEXT NOT NULL PRIMARY KEY,
        active BOOLEAN,
        "createdDate" DATETIME,
        "lastLogin" DATETIME,
        role TEXT,
        "signUpSource" TEXT,
        state TEXT
);
CREATE INDEX IF NOT EXISTS ix_users_index ON users ("index");


CREATE TABLE IF NOT EXISTS items (
        "index" BIGINT,
        barcode TEXT,
        description TEXT,
        "finalPrice" FLOAT,
        "itemPrice" FLOAT,
        "needsFetchReview" BOOLEAN,
        "partnerItemId" TEXT,
        "preventTargetGapPoints" BOOLEAN,
        "quantityPurchased" BIGINT,
        "userFlaggedBarcode" TEXT,
        "userFlaggedNewItem" BOOLEAN,
        "userFlaggedPrice" FLOAT,
        "userFlaggedQuantity" BIGINT,
        receipt_id TEXT,
        "needsFetchReviewReason" TEXT,
        "pointsNotAwardedReason" TEXT,
        "pointsPayerId" TEXT,
        "rewardsGroup" TEXT,
        "rewardsProductPartnerId" TEXT,
        "userFlaggedDescription" TEXT,
        "originalMetaBriteBarcode" TEXT,
        "originalMetaBriteDescription" TEXT,
        "brandCode" TEXT,
        "competitorRewardsGroup" TEXT,
        "discountedItemPrice" FLOAT,
        "originalReceiptItemText" TEXT,
        "itemNumber" TEXT,
        "originalMetaBriteQuantityPurchased" BIGINT,
        "pointsEarned" FLOAT,
        "targetPrice" FLOAT,
        "competitiveProduct" BOOLEAN,
        "originalFinalPrice" FLOAT,
        "originalMetaBriteItemPrice" TEXT,
        deleted BOOLEAN,
        "priceAfterCoupon" TEXT,
        "metabriteCampaignId" TEXT,
        FOREIGN KEY (receipt_id) REFERENCES receipts (receipt_id),
        FOREIGN KEY (barcode) REFERENCES brands (barcode),
        PRIMARY KEY (receipt_id,barcode)
);
CREATE INDEX IF NOT EXISTS ix_items_index ON items ("index");


CREATE TABLE IF NOT EXISTS brands (
        "index" BIGINT,
        barcode TEXT NOT NULL PRIMARY KEY,
        category TEXT,
        "categoryCode" TEXT,
        name TEXT,
        "topBrand" BOOLEAN,
        brand_id TEXT ,
        cpg_id TEXT,
        cpg_reference TEXT,
        "brandCode" TEXT
);
CREATE INDEX IF NOT EXISTS ix_brands_index ON brands ("index");

 * sqlite:///fetch_rewards.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### So, now we have 4 temp tables (not related) with data. And 4 tables (related) with no data yet. Now we just have to copy data from from temp tables to new related tables.

In [36]:
%%sql
SELECT name FROM sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///fetch_rewards.db
Done.


name
temp_receipts
temp_items
temp_users
temp_brands
receipts
users
items
brands


#### Disabling relationships to load data 

In [37]:
%%sql
PRAGMA foreign_keys = 0;

 * sqlite:///fetch_rewards.db
Done.


[]

In [38]:
%%sql
INSERT OR IGNORE INTO receipts SELECT * FROM temp_receipts;
INSERT OR IGNORE INTO users SELECT * FROM temp_users;
INSERT OR IGNORE INTO items SELECT * FROM temp_items;
INSERT OR IGNORE INTO brands SELECT * FROM temp_brands;

 * sqlite:///fetch_rewards.db
679 rows affected.
212 rows affected.
5753 rows affected.
1160 rows affected.


[]

#### Enabling relationships (Foreign Keys) and cheking the errors.

In [39]:
%%sql
PRAGMA foreign_keys = 1;
PRAGMA foreign_key_check(items);

 * sqlite:///fetch_rewards.db
Done.
Done.


table,rowid,parent,fkid
items,1,brands,0
items,2,brands,0
items,3,brands,0
items,5,brands,0
items,6,brands,0
items,7,brands,0
items,8,brands,0
items,10,brands,0
items,11,brands,0
items,12,brands,0


#### The most error-full table is "items". Let's see ho many empty references FK have in other tables (basically in a table "brands"), to be sure our SQL queries are returning at least somewhat meaningfull information.

In [40]:
result = %sql PRAGMA foreign_key_check(items);
df = result.DataFrame()
df.shape

 * sqlite:///fetch_rewards.db
Done.


(1842, 4)

In [41]:
%%sql
SELECT COUNT(rowid) from items;

 * sqlite:///fetch_rewards.db
Done.


COUNT(rowid)
5753


#### 1842 out of 5753 is not terribly bad considering dummy data. So, we will proceed with SQL queries from here.

#### Top 5 Brands by receipts scanned for the last 7 months. 
*I did 7 months, as 1 month haven't had any results.* 

In [83]:
%%sql
SELECT  brand_name,  DENSE_RANK () OVER (ORDER BY times_bought DESC) AS brand_rank
FROM (SELECT name brand_name, COUNT(*) times_bought FROM brands
      WHERE(  
                SELECT receipt_id FROM receipts
                WHERE dateScanned BETWEEN date('now','-7 month') AND date('now')
             )
      GROUP BY name)
LIMIT 5;

 * sqlite:///fetch_rewards.db
Done.


brand_name,brand_rank
Baken-Ets,1
Caleb's Kola,1
Diabetic Living Magazine,1
Dippin Dots® Cereal,1
Health Magazine,1
